In [7]:
import pandas as pd
from data_tweak.converters import convert_race, convert_sex
base_file_path = "../files"
out_file_path = base_file_path + "/events/officer_id_merged/stops/filter_v3"

officer_prof_df = pd.read_csv(base_file_path + "/profiles/officers_index_step_4.csv")
stops_df = pd.read_csv(base_file_path + "/events/stops_by_fo_birth_year.csv", index_col="stop_id")
stops_df.reset_index(inplace=True)

print("number of profiles:", officer_prof_df.shape[0])
officer_prof_df = officer_prof_df.dropna(subset=["off_year_of_birth"]).sort_values("off_year_of_birth")
print("number of profiles with a birth year:", officer_prof_df.shape[0])
print("number of stops:", stops_df.shape[0])

join_left_cols = ["first_off_first_name", "first_off_last_name", "first_off_sex", "first_off_race"]
join_right_cols = ["off_first_name", "off_last_name", "off_sex", "off_race"]
age_cols = ["first_off_age", "first_off_latest_birth_year"]

stops_df[stops_df[join_left_cols].isna().any(axis=1)].to_csv(out_file_path + "/first_off_key_missing.csv", index=False)

stops_df.dropna(axis="index", how="all", subset=join_left_cols, inplace=True)
stops_df.dropna(axis="index", how="any", subset=age_cols, inplace=True)
print("number of stops with a birth year:", stops_df.shape[0])
officer_prof_df.dropna(axis="index", how="all", subset=join_right_cols, inplace=True)
officer_prof_df.dropna(axis="index", how="any", subset=["off_year_of_birth"], inplace=True)

print("number of stops after dropping empty keys", stops_df.shape[0])
print("number of officers after dropping empty keys", officer_prof_df.shape[0])

convert_sex(stops_df, ["first_off_sex", "second_off_sex"])
convert_race(stops_df, ["first_off_race", "second_off_race"])

# Just do in the profile generation
convert_sex(officer_prof_df, ["off_sex"])
convert_race(officer_prof_df, ["off_race"])


/var/folders/9h/xh9hskwj4q9bvjqcy131pv9c0000gn/T/ipykernel_79813/1571582512.py:7: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  stops_df = pd.read_csv(base_file_path + "/events/stops_by_fo_birth_year.csv", index_col="stop_id")


number of profiles: 36504
number of profiles with a birth year: 36502
number of stops: 2865566
number of stops with a birth year: 2854561
number of stops after dropping empty keys 2854561
number of officers after dropping empty keys 36502


In [8]:

out_cols = list(stops_df.columns.values.tolist()) + ["off_uniq_id"]
stops_merged_df = pd.merge_asof(stops_df, officer_prof_df, left_on=["first_off_latest_birth_year"], right_on=["off_year_of_birth"], left_by=join_left_cols, right_by=join_right_cols, tolerance=1)
stops_merged_df.loc[:, out_cols].to_csv(out_file_path + "/half_merged.csv", index=False)
print("initial merge rows", stops_merged_df.shape[0])
print("rows with an uid for the first officer", stops_merged_df[stops_merged_df["off_uniq_id"].notna()].shape[0])


initial merge rows 2854561
rows with an uid for the first officer 2853653


In [9]:
join_left_cols = ["second_off_first_name", "second_off_last_name", "second_off_sex", "second_off_race"]
age_cols = ["second_off_age", "second_off_latest_birth_year"]
""" @TODO output rejected second off rows """
stops_merged_df[stops_merged_df[join_left_cols + age_cols].isna().any(axis=1)].to_csv(out_file_path + "/second_off_key_missing.csv", index=False)

In [10]:

second_off_stops = stops_merged_df.sort_values("second_off_latest_birth_year").dropna(subset=join_left_cols, how="any")
print("processing stops with a second officer", second_off_stops.shape[0])
second_off_stops_merged = pd.merge_asof(second_off_stops, officer_prof_df, left_on=["second_off_latest_birth_year"], right_on=["off_year_of_birth"], left_by=join_left_cols, right_by=join_right_cols, suffixes=("", "_second_off"), tolerance=1)
out_cols = list(stops_df.columns.values.tolist()) + ["off_uniq_id", "off_uniq_id_second_off"]
second_off_stops_merged.loc[:, out_cols].to_csv(out_file_path + "/second_off_stops_merged.csv", index=False)
print("number of stops matched with a second officer", second_off_stops_merged[second_off_stops_merged["off_uniq_id_second_off"].notna()].shape[0])


processing stops with a second officer 2464906
number of stops matched with a second officer 2463846


In [14]:
""" Consider, this chooses the closest match looking backwards in terms of birth date. Is this what we want? """

full_merge = pd.concat([second_off_stops_merged, stops_merged_df])
full_merge = full_merge.drop_duplicates(subset=["stop_id"])
full_merge.loc[:, out_cols].to_csv(out_file_path + "/full_merge.csv", index=False)
print("Total number of rows in the full merge", full_merge.shape[0])
print("Total number of rows missing a UID for the first officer", full_merge[full_merge["off_uniq_id"].isna()].shape[0])
missing_fo = full_merge[full_merge["off_uniq_id"].isna()]
missing_fo.to_csv(out_file_path + "/missing_fo_id.csv", index=False)
print("Total number of rows missing a UID for the second officer", full_merge[full_merge["second_off_first_name"].notna() & full_merge["off_uniq_id_second_off"].isna()].shape[0])
missing_so = full_merge[full_merge["second_off_first_name"].notna() & full_merge["off_uniq_id_second_off"].isna()]
missing_so.to_csv(out_file_path + "/missing_so_id.csv", index=False)



Total number of rows in the full merge 2854561
Total number of rows missing a UID for the first officer 908
Total number of rows missing a UID for the second officer 5113


In [15]:
print("Total number of rows missing a UID for just the second officer", full_merge[full_merge["off_uniq_id"].notna() & full_merge["second_off_first_name"].notna() & full_merge["off_uniq_id_second_off"].isna()].shape[0])

Total number of rows missing a UID for just the second officer 5113


In [19]:
""" attempt to match without race """

join_left_cols = ["first_off_first_name", "first_off_last_name", "first_off_sex"]
join_right_cols = ["off_first_name", "off_last_name", "off_sex"]
missing_fo.sort_values(by="first_off_latest_birth_year", inplace=True)

merged_without_race = pd.merge_asof(missing_fo, officer_prof_df, left_on=["first_off_latest_birth_year"], right_on=["off_year_of_birth"], left_by=join_left_cols, right_by=join_right_cols, tolerance=1)
merged_without_race

/var/folders/9h/xh9hskwj4q9bvjqcy131pv9c0000gn/T/ipykernel_79813/3405568228.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_fo.sort_values(by="first_off_latest_birth_year", inplace=True)


,stop_id,date_time,stop_address,district,beat,contact_type,first_off_last_name,first_off_first_name,first_off_sex,first_off_race,...,off_appointed_y,off_race_y,ranks_held_y,off_star_0_y,off_star_1_y,off_star_2_y,off_star_3_y,off_star_4_y,off_star_5_y,off_source_y
0,477358,2012-12-01 22:54:00+00:00,28XX DIVISION ST,13.0,1311.0,Suspicious Person,CHRIS,SYSTEM,NA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1108808,2013-09-29 10:57:00+00:00,41XX MICHIGAN AVE,2.0,213.0,Other,BRADLEY,VALENA,FEMALE,NaN,...,1989-12-26,BLACK,POLICE OFFICER,17810.0,10029.0,NaN,NaN,NaN,NaN,assignments_file
2,464193,2012-11-22 11:05:00+00:00,38XX CALUMET AVE,2.0,212.0,Other,BRADLEY,VALENA,FEMALE,NaN,...,1989-12-26,BLACK,POLICE OFFICER,17810.0,10029.0,NaN,NaN,NaN,NaN,assignments_file
3,62975,2012-02-18 12:27:00+00:00,2XX 35TH ST,21.0,2112.0,Traffic Related,BRADLEY,VALENA,FEMALE,NaN,...,1989-12-26,BLACK,POLICE OFFICER,17810.0,10029.0,NaN,NaN,NaN,NaN,assignments_file
4,39650,2012-02-01 14:45:00+00:00,35XX GILES AVE,2.0,211.0,Traffic Related,BRADLEY,VALENA,FEMALE,NaN,...,1989-12-26,BLACK,POLICE OFFICER,17810.0,10029.0,NaN,NaN,NaN,NaN,assignments_file
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903,1239909,2014-03-11 17:25:00+00:00,90XX ASHLAND AVE,22.0,2221.0,Gang / Narcotics Related,ALVARADO,MATTHEW,MALE,WHITE,...,2006-11-27,HISPANIC,POLICE OFFICER ASSIGNED AS FIELD TRAINING OFFICER,10456.0,NaN,NaN,NaN,NaN,NaN,assignments_file
904,2757426,2019-10-31 21:43:00+00:00,52XX IRVING PARK RD,16.0,1634.0,INVSTG,MATTHEWS,DAVID,MALE,NaN,...,2019-02-19,HISPANIC,POLICE OFFICER,18030.0,NaN,NaN,NaN,NaN,NaN,assignments_file
905,2863711,2020-12-19 22:35:00+00:00,38XX DIVISION ST,11.0,1112.0,INVSTG,MATTHEWS,DAVID,MALE,NaN,...,2019-02-19,HISPANIC,POLICE OFFICER,18030.0,NaN,NaN,NaN,NaN,NaN,assignments_file
906,2776344,2019-12-22 01:00:00+00:00,21XX LARAMIE AVE,25.0,2522.0,INVSTG,MATTHEWS,DAVID,MALE,NaN,...,2019-02-19,HISPANIC,POLICE OFFICER,18030.0,NaN,NaN,NaN,NaN,NaN,assignments_file
